In [1]:
import tensorflow as tf
import glob

In [2]:
def load_preprocess_image(path,labels):
    images = tf.io.read_file(path)
    images = tf.image.decode_jpeg(images,channels=3)
    images = tf.image.resize(images,[224,224])
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_flip_up_down(images)
    images = tf.cast(images,tf.float32)
    images = images/255.0
    labels = tf.reshape(labels,[1])
    return images,labels

In [3]:
def load_preprocess_image_test (path,labels):
    images = tf.io.read_file(path)
    images = tf.image.decode_jpeg(images,channels=3)
    images = tf.image.resize(images,[224,224])
    images = tf.cast(images,tf.float32)
    images = images/255.0
    labels = tf.reshape(labels,[1])
    return images,labels

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
train_image_path = glob.glob('../input/cat-and-dog/training_set/training_set/*/*.jpg')
train_image_label = [int(p.split('/')[5] == 'cats') for p in train_image_path]
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label))
train_image_ds = train_image_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)

test_image_path = glob.glob('../input/cat-and-dog/test_set/test_set/*/*.jpg')
test_image_label = [int(p.split('/')[5] == 'cats') for p in test_image_path]
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path,test_image_label))
test_image_ds = test_image_ds.map(load_preprocess_image_test,num_parallel_calls=AUTOTUNE)

In [6]:
BATCH_SIZE = 32
train_count = len(train_image_path)
test_count=len(test_image_path)
num_epochs = 30

train_image_ds = train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)
test_image_ds = test_image_ds.batch(BATCH_SIZE)
test_image_ds = test_image_ds.prefetch(AUTOTUNE)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,[3,3],input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64,[3,3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(128,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(256,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(256,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(256,(1,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(512,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(512,(3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(512,(1,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.GlobalAvgPool2D(),
    tf.keras.layers.Dense(2048),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
steps_per_epoch = train_count // BATCH_SIZE
validation_steps = test_count // BATCH_SIZE

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=['acc']
)
model.fit(
    train_image_ds,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_image_ds,
    validation_steps=validation_steps
)